In [1]:
# Install dependencies (if needed)
!pip install pandas numpy shapely --quiet

import pandas as pd
import numpy as np
from shapely.geometry import box

# -----------------------------
# Example Inputs
# -----------------------------

# CAD parts data (bounding box dimensions in meters)
# Normally extracted from CAD parsing module
parts_data = pd.DataFrame([
    {'part_id': 'P1', 'length': 2.0, 'width': 1.0, 'height': 0.5, 'quantity': 10},
    {'part_id': 'P2', 'length': 1.5, 'width': 1.5, 'height': 0.7, 'quantity': 5},
    {'part_id': 'P3', 'length': 0.5, 'width': 0.5, 'height': 0.3, 'quantity': 20},
])

# Workstation / machine footprint (length x width in meters)
workstations = pd.DataFrame([
    {'station_id': 'WS1', 'length': 3.0, 'width': 2.0},
    {'station_id': 'WS2', 'length': 2.5, 'width': 2.0},
])

# -----------------------------
# Step 1: Calculate individual part floor footprint
# -----------------------------
def part_floor_area(length, width, quantity):
    return length * width * quantity

parts_data['floor_area'] = parts_data.apply(
    lambda row: part_floor_area(row['length'], row['width'], row['quantity']), axis=1
)

total_parts_floor_area = parts_data['floor_area'].sum()
print(f"Total floor area required for parts: {total_parts_floor_area:.2f} m²")

# -----------------------------
# Step 2: Sum workstation footprints
# -----------------------------
total_workstation_area = (workstations['length'] * workstations['width']).sum()
print(f"Total floor area required for workstations: {total_workstation_area:.2f} m²")

# -----------------------------
# Step 3: Total required floor space
# -----------------------------
total_floor_space_required = total_parts_floor_area + total_workstation_area
print(f"Total estimated floor space required: {total_floor_space_required:.2f} m²")

# -----------------------------
# Step 4 (Optional): Optimize layout
# -----------------------------
# Simple 1D packing approximation (stack parts in rows to minimize floor space)
parts_data = parts_data.sort_values(by='floor_area', ascending=False)

layout_rows = []
current_row_length = 0
row_max_width = 0
max_width_limit = 10  # Example factory width limit in meters

for idx, row in parts_data.iterrows():
    if current_row_length + row['length'] <= max_width_limit:
        current_row_length += row['length']
        row_max_width = max(row['width'], row_max_width)
    else:
        layout_rows.append({'row_length': current_row_length, 'row_width': row_max_width})
        current_row_length = row['length']
        row_max_width = row['width']

# Append last row
layout_rows.append({'row_length': current_row_length, 'row_width': row_max_width})

optimized_floor_space = sum([r['row_length'] * r['row_width'] for r in layout_rows]) + total_workstation_area
print(f"Optimized estimated floor space: {optimized_floor_space:.2f} m²")


Total floor area required for parts: 36.25 m²
Total floor area required for workstations: 11.00 m²
Total estimated floor space required: 47.25 m²
Optimized estimated floor space: 17.00 m²
